In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys

from importlib import reload


sys.path.append("D:/DST/")
import cathay_db as db
import utils as ut
import financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

# 1. Read data from feather file

In [2]:
# # following is data from tej
# df_adjusted_price = pd.read_feather('data/tej_adjusted_price.feather')
# df_cash_dividends = pd.read_feather('data/tej_cash_dividends.feather')
# df_shareholder_meeting = pd.read_feather('data/tej_shareholder_meeting.feather')
# df_capital_formation = pd.read_feather('data/tej_capital_formation.feather')

# # following is data from cmoney
# df_price = pd.read_feather('data/cmoney_price.feather')
# df_company_info = pd.read_feather('data/cmoney_company_info.feather')
# df_financial_statement = pd.read_feather('data/cmoney_financial_statement.feather')

# ticker_list = np.sort(pd.unique(df_company_info['股票代號']))

In [3]:
# df_price = ut.cmoney_data_clean_up(df_price)
# df_financial_statement = ut.cmoney_data_clean_up(df_financial_statement)

# df_price = deepcopy(df_price[df_price['股票代號'].isin(ticker_list)])
# df_financial_statement = deepcopy(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)])

# df_adjusted_price = ut.tej_data_clean_up(df_adjusted_price)
# df_cash_dividends = ut.tej_data_clean_up(df_cash_dividends)
# df_shareholder_meeting = ut.tej_data_clean_up(df_shareholder_meeting)
# df_capital_formation = ut.tej_data_clean_up(df_capital_formation)

# df_adjusted_price = deepcopy(df_adjusted_price[df_adjusted_price['股票代號'].isin(ticker_list)])
# df_cash_dividends = deepcopy(df_cash_dividends[df_cash_dividends['股票代號'].isin(ticker_list)])
# df_shareholder_meeting = deepcopy(df_shareholder_meeting[df_shareholder_meeting['股票代號'].isin(ticker_list)])
# df_capital_formation = deepcopy(df_capital_formation[df_capital_formation['股票代號'].isin(ticker_list)])

In [4]:
# # save upper df to feather
# df_price = df_price.reset_index(drop=True)
# df_price.to_feather('data/df_price.feather')
# df_company_info.to_feather('data/df_company_info.feather')

# duplicates = df_financial_statement.columns[df_financial_statement.columns.duplicated()]
# new_columns = [f"{col}_{i}" if col in duplicates else col for i, col in enumerate(df_financial_statement.columns)]
# df_financial_statement.columns = new_columns

# df_financial_statement = df_financial_statement.reset_index(drop=True)
# df_financial_statement.to_feather('data/df_financial_statement.feather')

# df_adjusted_price = df_adjusted_price.reset_index(drop=True)
# df_adjusted_price.to_feather('data/df_adjusted_price.feather')

# df_cash_dividends = df_cash_dividends.reset_index(drop=True)
# df_cash_dividends.to_feather('data/df_cash_dividends.feather')

# df_shareholder_meeting = df_shareholder_meeting.reset_index(drop=True)
# df_shareholder_meeting.to_feather('data/df_shareholder_meeting.feather')

# df_capital_formation = df_capital_formation.reset_index(drop=True)
# df_capital_formation.to_feather('data/df_capital_formation.feather')

In [5]:
# read from feather
df_price = pd.read_feather('data/df_price.feather')
df_company_info = pd.read_feather('data/df_company_info.feather')
df_financial_statement = pd.read_feather('data/df_financial_statement.feather')
df_adjusted_price = pd.read_feather('data/df_adjusted_price.feather')
df_cash_dividends = pd.read_feather('data/df_cash_dividends.feather')
df_shareholder_meeting = pd.read_feather('data/df_shareholder_meeting.feather')
df_capital_formation = pd.read_feather('data/df_capital_formation.feather')


In [6]:
# read data from feather which is from main_01_data_cleaning.ipynb
df_price_liquidity_flag = pd.read_feather('./data/df_price_liquidity_flag.feather')
df_book_capital_melt = pd.read_feather('./data/df_book_capital_melt.feather')
df_price_dividend_slice = pd.read_feather('./data/df_price_dividend_slice.feather')
df_price_dividend = pd.read_feather('./data/df_price_dividend.feather')
df_share_outstanding = pd.read_feather('./data/df_share_outstanding.feather')

In [10]:
ticker_list = np.sort(pd.unique(df_company_info['股票代號']))

# Q5 Factors: 1. Market Factor
## 1.1 Market Factor: Stock Excess Return

In [7]:
df_risk_free_rate = pd.read_excel('./data/risk_free_rate.xlsx')

# change the column name to english
df_risk_free_rate.columns = ['date', 'risk_free_rate']

# change the date format, from 2023M05 to datetime format YYYY-MM
df_risk_free_rate['date'] = df_risk_free_rate['date'].apply(lambda x: datetime.datetime.strptime(x, '%YM%m').strftime('%Y-%m'))
# shift the date to next month and create a data valid date for it
df_risk_free_rate['data_valid_date'] = df_risk_free_rate['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m') + datetime.timedelta(days=31))

# sort the data_valid_date for ffill
df_risk_free_rate = df_risk_free_rate.sort_values(by=['data_valid_date']).reset_index(drop=True)

# divide the risk_free_rate by 100
df_risk_free_rate['risk_free_rate'] = df_risk_free_rate['risk_free_rate'] / 100

# select the data_valid_date and risk_free_rate
df_risk_free_rate = df_risk_free_rate[['data_valid_date', 'risk_free_rate']]

# change the data to daily data
df_risk_free_rate = df_risk_free_rate.set_index('data_valid_date').resample('D').ffill()

# change the annualized risk_free_rate to monthly risk_free_rate
df_risk_free_rate['risk_free_rate'] = (1 + df_risk_free_rate['risk_free_rate']) ** (1/12) - 1

In [8]:
df_excess_return = df_adjusted_price[['年月日', '股票代號', '股票名稱', '收盤價(元)']].reset_index(drop=True).copy()
# change the column name
df_excess_return.columns = ['date', 'ticker', 'name', 'price']

# change the price to float
df_excess_return['price'] = df_excess_return['price'].astype(float)

# groupby the ticker and calculate 20 days return by pct_change
df_excess_return['20_d_return'] = df_excess_return.groupby('ticker')['price'].pct_change(20)

# merge the risk_free_rate to df_excess_return
df_excess_return = df_excess_return.merge(df_risk_free_rate, how='left', left_on='date', right_on='data_valid_date')

# calculate the excess return by 20 days return - risk_free_rate
df_excess_return['excess_return'] = df_excess_return['20_d_return'] - df_excess_return['risk_free_rate']

df_excess_return.dropna(inplace=True)

df_excess_return = df_excess_return.reset_index(drop=True)

# fs.CDF(df_excess_return).find_ticker('2330').tail(30)

## 1.2 Investment Factor:
### 1.2.1 Investment Factor: Asset Growth

In [11]:
df_asset_melt = fs.quarterly_to_daily(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)], '資產總計(千)')
df_asset_melt.columns = ['date', 'ticker', 'asset']
# # calculate the yoy and qoq
# df_asset_melt['asset_yoy'] = df_asset_melt.groupby('ticker', group_keys=False, as_index=False)['asset'].pct_change(252)
# df_asset_melt['asset_qoq'] = df_asset_melt.groupby('ticker', group_keys=False, as_index=False)['asset'].pct_change(63)

df_asset_qoq = fs.calc_periodic(df_financial_statement, '資產總計(千)','qoq')
df_asset_qoq_melt = fs.quarterly_to_daily(df_asset_qoq, '資產總計(千)_qoq')
df_asset_yoy = fs.calc_periodic(df_financial_statement, '資產總計(千)','yoy')
df_asset_yoy_melt = fs.quarterly_to_daily(df_asset_yoy, '資產總計(千)_yoy')

In [21]:
df_asset_qoq

,股票代號,年季,建立日期,資產總計(千)_qoq
0,1101,199001,20151209,NaN
1,1101,199002,20151209,0.9516
2,1101,199003,20151209,1.1221
3,1101,199004,20151209,0.8778
4,1101,199101,20151209,1.0606
...,...,...,...,...
141404,9962,202201,20220510,0.8870
141405,9962,202202,20220810,1.0656
141406,9962,202203,20221110,0.9151
141407,9962,202204,20230323,1.0273


### 1.2.2 Investment Factor: PPE Growth

In [8]:
# df_ppe_melt = fs.quarterly_to_daily(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)], '不動產、廠房及設備(千)')
# df_ppe_melt.columns = ['date', 'ticker', 'ppe']
# # calculate the yoy and qoq
# df_ppe_melt['ppe_yoy'] = df_ppe_melt.groupby('ticker', group_keys=False, as_index=False)['ppe'].pct_change(252)
# df_ppe_melt['ppe_qoq'] = df_ppe_melt.groupby('ticker', group_keys=False, as_index=False)['ppe'].pct_change(63)

df_ppe_qoq = fs.calc_periodic(df_financial_statement, '不動產、廠房及設備(千)','qoq')
df_ppe_qoq_melt = fs.quarterly_to_daily(df_ppe_qoq, '不動產、廠房及設備(千)_qoq')
df_ppe_yoy = fs.calc_periodic(df_financial_statement, '不動產、廠房及設備(千)','yoy')
df_ppe_yoy_melt = fs.quarterly_to_daily(df_ppe_yoy, '不動產、廠房及設備(千)_yoy')

## 1.3 Profitability Factor: ROE

In [9]:
df_ni_qoq = fs.calc_periodic(df_financial_statement, '稅後純益(千)','qoq')
df_ni_qoq_melt = fs.quarterly_to_daily(df_ni_qoq, '稅後純益(千)_qoq')
df_ni_yoy = fs.calc_periodic(df_financial_statement, '稅後純益(千)','yoy')
df_ni_yoy_melt = fs.quarterly_to_daily(df_ni_yoy, '稅後純益(千)_yoy')

In [10]:
df_ni_melt = fs.quarterly_to_daily(df_financial_statement, '稅後純益(千)')
df_ni_melt.columns = ['date', 'ticker', 'ni']
# calculate the yoy and qoq
# df_ni_melt['ni_yoy'] = df_ni_melt.groupby('ticker', group_keys=False, as_index=False)['ni'].pct_change(252)
# df_ni_melt['ni_qoq'] = df_ni_melt.groupby('ticker', group_keys=False, as_index=False)['ni'].pct_change(63)

df_preferred_share_melt = fs.quarterly_to_daily(df_financial_statement, '特別股股本(千)')
df_preferred_share_melt.columns = ['date', 'ticker', 'preferred_share']

df_equity_melt = fs.quarterly_to_daily(df_financial_statement, '權益總計(千)')
df_equity_melt.columns = ['date', 'ticker', 'equity']

df_equity_melt = df_equity_melt.merge(df_preferred_share_melt, how='left', on=['date', 'ticker'])
df_equity_melt['preferred_share'] = df_equity_melt['preferred_share'].fillna(0)
df_equity_melt['equity - preferred_share'] = df_equity_melt['equity'] - df_equity_melt['preferred_share']
df_roe_melt = df_ni_melt.merge(df_equity_melt[['date', 'ticker', 'equity - preferred_share']], how='left', on=['date', 'ticker']).copy()
df_roe_melt['roe'] = df_roe_melt['ni'] / (df_roe_melt['equity - preferred_share'] / 4)

In [11]:
accounting_item = '稅後純益(千)'

df = df_financial_statement[['年季','股票代號','建立日期', accounting_item]].copy()

# drop na
df.dropna(inplace=True)

# drop the duplicate
df.drop_duplicates(inplace=True)

# change the accounting_item to float
df[accounting_item] = df[accounting_item].astype(float)

df = df.sort_values(by=['股票代號', '年季']).reset_index(drop=True)

df[f'{accounting_item}_4Q_sum'] = df.groupby('股票代號', as_index=False, group_keys=False).apply(lambda x: x[accounting_item].rolling(4).sum())

df_ni_4q_sum_melt = fs.quarterly_to_daily(df, '稅後純益(千)_4Q_sum')
df_ni_4q_sum_melt.columns = ['date', 'ticker', 'ni_4Q_sum']

In [12]:
df_roe_4q_sum_melt = df_ni_4q_sum_melt.merge(df_equity_melt[['date', 'ticker', 'equity - preferred_share']], how='left', on=['date', 'ticker']).copy()
df_roe_4q_sum_melt['roe_4q_sum'] = df_roe_4q_sum_melt['ni_4Q_sum'] / df_roe_4q_sum_melt['equity - preferred_share'] 

## 1.4 Tobin's Q

In [13]:
df_market_cap_melt = pd.merge(df_price[['日期','股票代號','收盤價']], df_share_outstanding, how='left', left_on=['日期','股票代號'], right_on=['日期','股票代號'])
df_market_cap_melt['收盤價'] = df_market_cap_melt['收盤價'].astype(float)
df_market_cap_melt['流通在外股數'] = df_market_cap_melt['流通在外股數'].astype(float)
df_market_cap_melt.columns = ['date', 'ticker', 'price', 'share_outstanding']

# the market cap is the product of price and share outstanding * 1000(share) / 1000(unit)
df_market_cap_melt['market_cap'] = df_market_cap_melt['price'] * df_market_cap_melt['share_outstanding']
df_market_cap_melt.sort_values(by=['ticker', 'date'], inplace=True)
df_market_cap_melt = df_market_cap_melt.reset_index(drop=True)

In [14]:
df_long_term_liability_melt = fs.quarterly_to_daily(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)], '長期負債合計(千)')
df_long_term_liability_melt.columns = ['date', 'ticker', 'long_term_liability']

df_short_term_liability_melt = fs.quarterly_to_daily(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)], '短期借款(千)')
df_short_term_liability_melt.columns = ['date', 'ticker', 'short_term_liability']

df_tobins_q_melt = df_market_cap_melt.merge(df_asset_melt, how='left', on=['date', 'ticker']).merge(df_long_term_liability_melt, how='left', on=['date', 'ticker']).merge(df_short_term_liability_melt, how='left', on=['date', 'ticker'])
df_tobins_q_melt['long_term_liability'] = df_tobins_q_melt['long_term_liability'].fillna(0)
df_tobins_q_melt['short_term_liability'] = df_tobins_q_melt['short_term_liability'].fillna(0)


df_tobins_q_melt['tobins_q'] = (df_tobins_q_melt['market_cap'] + df_tobins_q_melt['long_term_liability'] + df_tobins_q_melt['short_term_liability']) / df_tobins_q_melt['asset']

In [17]:
df_tobins_q_melt[df_tobins_q_melt['tobins_q'].isna()].sort_values(['date','ticker'])

,date,ticker,price,share_outstanding,market_cap,asset,long_term_liability,short_term_liability,tobins_q
23252,2000-01-04,1107,9.1500,NaN,NaN,NaN,0.0000,0.0000,NaN
43742,2000-01-04,1201,8.1500,NaN,NaN,14727709.0000,1509659.0000,2155815.0000,NaN
55200,2000-01-04,1204,18.0000,NaN,NaN,NaN,0.0000,0.0000,NaN
56606,2000-01-04,1206,18.5000,NaN,NaN,NaN,0.0000,0.0000,NaN
56697,2000-01-04,1207,9.6000,NaN,NaN,NaN,0.0000,0.0000,NaN
...,...,...,...,...,...,...,...,...,...
7933257,2023-06-26,9951,75.9000,74900.0000,5684910.0000,NaN,0.0000,0.0000,NaN
7938060,2023-06-26,9955,24.1500,103208.2000,2492478.0300,NaN,0.0000,0.0000,NaN
7941826,2023-06-26,9958,112.5000,232396.4000,26144595.0000,NaN,0.0000,0.0000,NaN
7946397,2023-06-26,9960,29.9000,33592.5000,1004415.7500,NaN,0.0000,0.0000,NaN


## 1.5 OCF

In [22]:
df_ocf_melt = fs.quarterly_to_daily(df_financial_statement, '營業活動現金流量累計(千)')
df_ocf_melt.columns = ['date', 'ticker', 'ocf']

# merge asset to df_ocf_melt
df_ocf_melt = df_ocf_melt.merge(df_asset_melt, how='left', on=['date', 'ticker'])
df_ocf_melt['ocf / asset'] = df_ocf_melt['ocf'] / df_ocf_melt['asset']

In [34]:
list_delisted = pd.unique(pd.read_feather(r'D:\Projects\0_HIDIV\data\下市櫃公司基本資料\cmoney_data.feather')['股票代號'])

In [42]:
list_delisted[:5]

array(['6247', '6594', '1507', '1592', '2456'], dtype=object)

In [41]:
pd.unique(df_asset_melt[(df_asset_melt['ticker'].isin(list_delisted)) & (df_asset_melt['asset'].isna() == False)]['ticker'])
# ut.CDF(df_asset_melt[df_asset_melt['ocf'].isna()].sort_values(['date','ticker'])).find_ticker('1107')

array(['2432'], dtype=object)

## 1.6 dROE

In [16]:
df_roe_melt['roe_yoy'] = df_roe_melt.groupby('ticker', group_keys=False, as_index=False)['roe'].pct_change(252)
# 

In [17]:
df_roe_4q_sum_melt['roe_4q_sum_yoy'] = df_roe_4q_sum_melt.groupby('ticker', group_keys=False, as_index=False)['roe_4q_sum'].pct_change(252)

# Factor df to feather file 

In [18]:
df_excess_return.to_feather('./data/df_excess_return.feather')
df_asset_qoq_melt.to_feather('./data/df_asset_qoq_melt.feather')
df_asset_yoy_melt.to_feather('./data/df_asset_yoy_melt.feather')
df_ppe_qoq_melt.to_feather('./data/df_ppe_qoq_melt.feather')
df_ppe_yoy_melt.to_feather('./data/df_ppe_yoy_melt.feather')
df_ni_qoq_melt.to_feather('./data/df_ni_qoq_melt.feather')
df_ni_yoy_melt.to_feather('./data/df_ni_yoy_melt.feather')
df_roe_melt.to_feather('./data/df_roe_melt.feather')
df_roe_4q_sum_melt.to_feather('./data/df_roe_4q_sum_melt.feather')
df_tobins_q_melt.to_feather('./data/df_tobins_q_melt.feather')
df_ocf_melt.to_feather('./data/df_ocf_melt.feather')

In [6]:
# read data from feather which is from main_02_factor.ipynb
df_excess_return = pd.read_feather('./data/df_excess_return.feather')
df_asset_qoq_melt = pd.read_feather('./data/df_asset_qoq_melt.feather')
df_asset_yoy_melt = pd.read_feather('./data/df_asset_yoy_melt.feather')
df_ppe_qoq_melt = pd.read_feather('./data/df_ppe_qoq_melt.feather')
df_ppe_yoy_melt = pd.read_feather('./data/df_ppe_yoy_melt.feather')
df_ni_qoq_melt = pd.read_feather('./data/df_ni_qoq_melt.feather')
df_ni_yoy_melt = pd.read_feather('./data/df_ni_yoy_melt.feather')
df_roe_melt = pd.read_feather('./data/df_roe_melt.feather')
df_roe_4q_sum_melt = pd.read_feather('./data/df_roe_4q_sum_melt.feather')
df_tobins_q_melt = pd.read_feather('./data/df_tobins_q_melt.feather')
df_ocf_melt = pd.read_feather('./data/df_ocf_melt.feather')

# Merge the factors to one df

In [19]:
df_factor_all = df_adjusted_price[['年月日','股票代號','收盤價(元)']].copy()
df_factor_all.columns = ['date', 'ticker', 'price']
# change the date to datetime format
df_factor_all['date'] = pd.to_datetime(df_factor_all['date'], format='%Y%m%d')
# change the price to float
df_factor_all['price'] = df_factor_all['price'].astype(float)

In [20]:
# merge the df_excess_return to df_factor_all
df_factor_all = df_factor_all.merge(df_excess_return[['date', 'ticker', 'excess_return']], how='left', on=['date', 'ticker'])

df_asset_qoq_melt.columns = ['date', 'ticker', 'asset_qoq']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_asset_qoq_melt, how='left', on=['date', 'ticker'])

df_asset_yoy_melt.columns = ['date', 'ticker', 'asset_yoy']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_asset_yoy_melt, how='left', on=['date', 'ticker'])

df_ppe_qoq_melt.columns = ['date', 'ticker', 'ppe_qoq']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ppe_qoq_melt, how='left', on=['date', 'ticker'])

df_ppe_yoy_melt.columns = ['date', 'ticker', 'ppe_yoy']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ppe_yoy_melt, how='left', on=['date', 'ticker'])

df_ni_qoq_melt.columns = ['date', 'ticker', 'ni_qoq']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ni_qoq_melt, how='left', on=['date', 'ticker'])

df_ni_yoy_melt.columns = ['date', 'ticker', 'ni_yoy']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ni_yoy_melt, how='left', on=['date', 'ticker'])

df_roe_melt = df_roe_melt[['date', 'ticker', 'roe', 'roe_yoy']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_roe_melt, how='left', on=['date', 'ticker'])

df_roe_4q_sum_melt = df_roe_4q_sum_melt[['date', 'ticker', 'roe_4q_sum', 'roe_4q_sum_yoy']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_roe_4q_sum_melt, how='left', on=['date', 'ticker'])

df_tobins_q_melt = df_tobins_q_melt[['date', 'ticker', 'tobins_q']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_tobins_q_melt, how='left', on=['date', 'ticker'])

df_ocf_melt = df_ocf_melt[['date', 'ticker', 'ocd / asset']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ocf_melt, how='left', on=['date', 'ticker'])

In [21]:
df_price_dividend['dividend_1Y_sum_yield'] = df_price_dividend['DIV_fillna_0_1_Y_sum'] / df_price_dividend['收盤價']
df_price_dividend['dividend_2Y_sum_yield'] = df_price_dividend['DIV_fillna_0_2_Y_sum'] / df_price_dividend['收盤價']
df_price_dividend['dividend_3Y_sum_yield'] = df_price_dividend['DIV_fillna_0_3_Y_sum'] / df_price_dividend['收盤價']

df_price_dividend = df_price_dividend[['日期','股票代號','dividend_1Y_sum_yield','dividend_2Y_sum_yield','dividend_3Y_sum_yield']].copy()

df_price_dividend.columns = ['date', 'ticker', 'dividend_1Y_sum_yield', 'dividend_2Y_sum_yield', 'dividend_3Y_sum_yield']

df_factor_all = pd.merge(df_factor_all, df_price_dividend, how='left', on=['date', 'ticker'])

In [22]:
df_price_dividend_slice.columns = ['date', 'ticker', 'last_dividend_yield']

df_factor_all = pd.merge(df_factor_all, df_price_dividend_slice, how='left', on=['date', 'ticker'])

In [23]:
df_factor_all.sort_values(by=['ticker', 'date'], inplace=True)

In [24]:
df_factor_all = df_factor_all.reset_index(drop=True)
df_factor_all.to_feather('./data/df_factor_all.feather')

In [25]:
df_factor_all

,date,ticker,price,excess_return,asset_qoq,asset_yoy,ppe_qoq,ppe_yoy,ni_qoq,ni_yoy,roe,roe_yoy,roe_4q_sum,roe_4q_sum_yoy,tobins_q,ocd / asset,dividend_1Y_sum_yield,dividend_2Y_sum_yield,dividend_3Y_sum_yield,last_dividend_yield
0,2000-01-04,1101,7.5067,NaN,1.0098,1.0897,1.0210,1.0869,0.3967,0.3842,0.0304,-0.7894,0.0646,-0.2398,1.2977,0.0130,NaN,NaN,NaN,NaN
1,2000-01-05,1101,7.7020,NaN,1.0098,1.0897,1.0210,1.0869,0.3967,0.3842,0.0304,-0.7894,0.0646,-0.2398,1.3253,0.0130,NaN,NaN,NaN,NaN
2,2000-01-06,1101,7.3766,NaN,1.0098,1.0897,1.0210,1.0869,0.3967,0.3842,0.0304,-0.7894,0.0646,-0.2398,1.2792,0.0130,NaN,NaN,NaN,NaN
3,2000-01-07,1101,7.3549,NaN,1.0098,1.0897,1.0210,1.0869,0.3967,0.3842,0.0304,-0.7894,0.0646,-0.2398,1.2762,0.0130,NaN,NaN,NaN,NaN
4,2000-01-10,1101,7.5935,NaN,1.0098,1.0897,1.0210,1.0869,0.3967,0.3842,0.0304,-0.7894,0.0646,-0.2398,1.3099,0.0130,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7905566,2023-04-24,9962,19.9500,-0.0147,1.0273,0.8885,1.0972,1.0850,0.8529,0.9260,0.1727,-0.2554,0.1771,-0.0166,1.2743,0.4152,0.0551,0.0551,0.0652,0.0000
7905567,2023-04-25,9962,19.2500,-0.0428,1.0273,0.8885,1.0972,1.0850,0.8529,0.9260,0.1727,-0.2554,0.1771,-0.0166,1.2320,0.4152,0.0571,0.0571,0.0675,0.0000
7905568,2023-04-26,9962,19.2000,-0.0206,1.0273,0.8885,1.0972,1.0850,0.8529,0.9260,0.1727,-0.2554,0.1771,-0.0166,1.2290,0.4152,0.0573,0.0573,0.0677,0.0000
7905569,2023-04-27,9962,19.2000,-0.0342,1.0273,0.8885,1.0972,1.0850,0.8529,0.9260,0.1727,-0.2554,0.1771,-0.0166,1.2290,0.4152,0.0573,0.0573,0.0677,0.0000
